In [1]:
import pandas as pd
import numpy as np 

# set_tdcsfog_dataset = pd.read_parquet('Data/Process/set_tdcsfog_data.parquet')
# test_tdcsfog_dataset = pd.read_parquet('Data/Process/test_tdcsfog_data.parquet')
# Test_tdcsfog_dataset = pd.read_csv('Data/test/tdcsfog/')

In [2]:
# def preprocessing_the_dataset(df):
#     def check_skewness(df):
#     # this can check relation between each column
#         skew_limit=0.75
#         skew_value=df[df.columns].skew()
#         #print(skew_value)
#         skew_col=skew_value[abs(skew_value)>skew_limit]
#         cols=skew_col.index
#         return cols

#     import random 
#     random_seed = 54
    
#     feature_col = ['AccV','AccML','AccAP']
#     label_col = ['StartHesitation','Turn','Walking', 'All_zero']

#     # make feature and label
#     feature_dataset = df[feature_col]
#     label_dataset = df[label_col]
    
#     # check skewness and powertransform
#     skew_columns = check_skewness(feature_dataset)
#     print(skew_columns)
    
#     print("Power Transform start")
#     from sklearn.preprocessing import PowerTransformer
#     pt=PowerTransformer(standardize=False)  
#     feature_dataset[skew_columns] = pt.fit_transform(feature_dataset[skew_columns])
    
#     print("Standardization start")
#     # Change features data to 0 and 1
#     from sklearn.preprocessing import StandardScaler
#     sc=StandardScaler()
#     feature_dataset=sc.fit_transform(feature_dataset)
    
#     print("Train test split begin")
#     from sklearn.model_selection import train_test_split
#     train_feature, valid_feature, train_label, valid_label = train_test_split(feature_dataset, label_dataset, test_size=0.2, random_state=True)
    
#     train_feature = np.array(train_feature) 
#     valid_feature = np.array(valid_feature)
#     train_label  = np.array(train_label)
#     valid_label = np.array(valid_label)
    
#     return train_feature, valid_feature, train_label, valid_label
#     # return feature_dataset, label_dataset
    

In [3]:
# train_feature, valid_feature, train_label, valid_label = preprocessing_the_dataset(set_tdcsfog_dataset)

In [4]:
# test_feature, test_label = preprocessing_the_dataset(test_tdcsfog_dataset)


In [5]:
# test_feature.shape, test_label.shape

In [6]:
# np.save('Data/tdcsfog/train_feature',train_feature)
# np.save('Data/tdcsfog/valid_feature',valid_feature)
# np.save('Data/tdcsfog/train_label',train_label)
# np.save('Data/tdcsfog/valid_label',valid_label)
# np.save('Data/tdcsfog/test_feature',test_feature)
# np.save('Data/tdcsfog/test_label',test_label)

#### bz of low memory, I save the features and label then load it

In [7]:
import pandas as pd
import numpy as np 

train_feature = np.load('Data/tdcsfog/train_feature.npy')
valid_feature = np.load('Data/tdcsfog/valid_feature.npy')
train_label = np.load('Data/tdcsfog/train_label.npy')
valid_label = np.load('Data/tdcsfog/valid_label.npy')
test_feature = np.load('Data/tdcsfog/test_feature.npy')
test_label = np.load('Data/tdcsfog/test_label.npy')

In [8]:
train_feature.shape, valid_feature.shape, train_label.shape, valid_label.shape

((12454643, 3), (3113661, 3), (12454643, 4), (3113661, 4))

### Build Model

In [9]:
# SKlearn model
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import  RadiusNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier, cv

# import lightgbm as lgb


In [10]:


#Building Model Dict
Models = {                         
    # "Decision Tree": DecisionTreeClassifier(),      
    "KNearest": KNeighborsClassifier(n_jobs=-1),           
    # "Ridge" : RidgeClassifier(),              #poor result    
    # "MLP" : MLPClassifier(),                  #poor result              
    # "R_Neighour" : RadiusNeighborsClassifier(),
    # "Extra_T" : ExtraTreesClassifier(n_estimators=50),
    # "R_forest" : RandomForestClassifier(),
    "XGB" : xgb.XGBClassifier(tree_method='gpu_hist', gpu_id=0),
    # "Catboost" : CatBoostClassifier()
    }



In [11]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report

In [12]:
import numpy as np
np.random.seed(40)
from mlflow import MlflowClient
from mlflow.entities import ViewType
from mlflow.models.signature import infer_signature

# !mlflow server --backend-store-uri sqlite:///backend.db

In [13]:
def eval_metrics(classifier, test_features, test_labels):
    
    # make prediction
    predictions   = classifier.predict(test_features)
    
    base_score   = classifier.score(test_features,test_labels)
    accuracy = accuracy_score(test_labels, predictions)
    av_precision = average_precision_score(test_labels, predictions)
 
    
    target_names = ['StartHesitation','Turn','Walking', 'All_zero']
    print("Classification report")
    print("---------------------","\n")
    print(classification_report(test_labels, predictions,target_names=target_names),"\n")
    
    print("Accuracy Measures")
    print("---------------------","\n")
    print("Base score: ", base_score)
    print("Accuracy: ", accuracy)
    print("Avarge Precision: ", av_precision)
    
    return base_score,accuracy,av_precision

In [14]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")


tracking URI: 'http://127.0.0.1:5000'


In [15]:
from sklearn.model_selection import cross_val_score
counter = 1
for Model_Name, classifier in Models.items(): 
    # with mlflow.start_run(nested=True):
    print(f"{counter}. {Model_Name}")
    
    with mlflow.start_run():
        # fit the model
        from joblib import parallel_backend

        with parallel_backend('threading', n_jobs=-1):
            classifier.fit(train_feature, train_label)
        # cross_val_scores = cross_val_score(classifier, train_feature, train_label, cv=3)
        # round_scores = round(cross_val_scores.mean(), 4) * 100
        # print("Classifiers: ", classifier.__class__.__name__, "Has a training score of", round_scores , "% accuracy score")

        
        counter = counter + 1
        
        # Calculate the metrics
        base_score,accuracy,av_precision = eval_metrics(classifier,
                                                        valid_feature,
                                                        valid_label)  
        
        mlflow.log_param("Model"           , Model_Name)
        mlflow.log_param("Dataset" , "Tdcsfog")
        mlflow.log_metric("base_score"     , base_score)
        mlflow.log_metric("accuracy"       , accuracy)
        mlflow.log_metric("av_precision"   , av_precision)
        # mlflow.log_metric("round_score", round_scores)
        
        # signature = infer_signature(valid_feature, classifier.predict(valid_feature))
        if av_precision > 0.95 :
            mlflow.sklearn.log_model(classifier,Model_Name)
            print(f"f1 socre is more than 0.945 so the {Model_Name} is saved")
        else :
            print(f"Because f1 socre is not quality. The model is skip to saving phase.")
        
        print("________________________________________")

1. KNearest


MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/runs/create failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/create (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7facff37e8d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
# # LGB
from sklearn.metrics import f1_score
params = {
          'boosting_type': 'gbdt',
          'objective': 'multiclass',
          'metric':'multi_logloss',
          'num_class':4,
          'learning_rate': 0.05,
          'verbose' : -1
         }

import lightgbm as lgb
lgb_train = lgb.Dataset(train_feature, label = train_label)
lgb_test = lgb.Dataset(valid_feature, label = valid_label)

In [ ]:
LGBM = lgb.train(params, lgb_train, 100)
LGBM_pred=LGBM.predict(test_feature)
LGBM_pred = LGBM_pred.round()

In [ ]:
LGBM_accuracy = accuracy_score(y_true=test_label,
                               y_pred=LGBM_pred,
                            #    average='weighted'
                            )
LGBM_av_precision = average_precision_score(
   y_true=test_label,
   y_score=LGBM_pred
)
print(LGBM_accuracy, LGBM_av_precision)


0.25020785822270686 0.24999999999999997


In [ ]:
# Catboost=CatBoostClassifier(eval_metric='Accuracy',use_best_model=True,random_seed=42,)
# Catboost.fit(train_feature,train_label,eval_set=(valid_feature,valid_label))

In [ ]:
# # Cat_pred = Catboost.predict(test_feature)
# base_score, accuracy, av_precision = eval_metrics(classifier = Castboost,
#                                       test_feature = test_feature,
#                                       test_label = test_label)

In [ ]:
for Model_Name, classifier in Models.items(): 
    print(classifier)
    base_score,accuracy,av_precision = eval_metrics(classifier=classifier,
                                                    test_features=test_feature,
                                                    test_labels= test_label)  
    

KNeighborsClassifier(n_jobs=-1)
Classification report
--------------------- 



/home/hanlinn/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                 precision    recall  f1-score   support

StartHesitation       0.80      0.87      0.83    973828
           Turn       0.71      0.65      0.68    973227
        Walking       0.81      0.88      0.84    973253
       All_zero       0.80      0.54      0.65    971768

      micro avg       0.78      0.74      0.76   3892076
      macro avg       0.78      0.74      0.75   3892076
   weighted avg       0.78      0.74      0.75   3892076
    samples avg       0.74      0.74      0.74   3892076
 

Accuracy Measures
--------------------- 

Base score:  0.7369912612189484
Accuracy:  0.7369912612189484
Avarge Precision:  0.6422217471842908
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=0, grow_policy=None, importance_type=None,

/home/hanlinn/.local/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
